In [2]:
# Initialize the workspace
from pymongo import MongoClient
import pprint
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [3]:
#Connect w/ Mongo 
client = MongoClient('localhost', 27017)
db = client['states']
#alaska = db['Alaska']

In [6]:
#alaska.insert_one({'test': 'testy'})

In [4]:
#attempt to scrape EIA webpage for states providing flaring / productioninfo

main_flare_url = 'https://www.eia.gov/dnav/ng/ng_prod_sum_a_EPG0_VGV_mmcf_a.htm'
r = requests.get(main_flare_url)

In [6]:
type(r)

requests.models.Response

In [7]:
r.status_code

200

In [9]:
pprint.pprint(r.text)

('<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">\r\n'
 '\r\n'
 '<html>\r\n'
 '<head>\r\n'
 '\t<title>Natural Gas Vented and Flared </title>\r\n'
 '\t<meta name="robots" content="index, follow">\r\n'
 '    <meta http-equiv="X-UA-Compatible" content="IE=9" />\r\n'
 '\t\t<link rel="StyleSheet" href="Styles/ng_wrapper3.css" TYPE="text/css">\r\n'
 '\t\t<link rel="StyleSheet" href="Styles/ng_data4.css" TYPE="text/css">\r\n'
 '\t\t<link rel="StyleSheet" href="Styles/Nav_Print.css" TYPE="text/css" '
 'media="print">\r\n'
 '\t    <link rel="stylesheet" href="/styles/Eia_sitewideF.css" '
 'type="text/css" />\t\t\t\t\r\n'
 '\t\t\r\n'
 ' <NOSCRIPT>\r\n'
 '\t  <H3>This page uses JavaScipt</H3>\r\n'
 '\t  <ul>\r\n'
 '\t  <li>Please use Internet Explorer 3+ or Netscape Navigator 3+\r\n'
 '\t  <li>Make sure that JavaScript is enabled in your browser\r\n'
 '\t  </ul>\r\n'
 '  </NOSCRIPT>\r\n'
 '  \r\n'
 '  <!-- Header Script -->\r\n'
 '    <script language="JavaScript" src="/styles/eia

In [12]:
soup = BeautifulSoup(r.text, 'html.parser')

In [14]:
print(soup.prettify())

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html>
 <head>
  <title>
   Natural Gas Vented and Flared
  </title>
  <meta content="index, follow" name="robots"/>
  <meta content="IE=9" http-equiv="X-UA-Compatible">
   <link href="Styles/ng_wrapper3.css" rel="StyleSheet" type="text/css"/>
   <link href="Styles/ng_data4.css" rel="StyleSheet" type="text/css"/>
   <link href="Styles/Nav_Print.css" media="print" rel="StyleSheet" type="text/css"/>
   <link href="/styles/Eia_sitewideF.css" rel="stylesheet" type="text/css">
    <noscript>
     <h3>
      This page uses JavaScipt
     </h3>
     <ul>
      <li>
       Please use Internet Explorer 3+ or Netscape Navigator 3+
       <li>
        Make sure that JavaScript is enabled in your browser
       </li>
      </li>
     </ul>
    </noscript>
    <!-- Header Script -->
    <script language="JavaScript" src="/styles/eia_header.js" type="text/javascript">
    </script>
    <!--/ Header Script -->
    <script src="/global/in

In [16]:
#Filter for each row
row = soup.find_all('tr', class_ = 'DataRow')[0]
row

<tr class="DataRow">
<td class="DataStub" width="228">
<table border="0" cellpadding="0" cellspacing="0" class="data2">
<tr>
<td width="3"></td>
<td class="DataStub1"><b>U.S.<b></b></b></td>
</tr>
</table>
</td>
<td class="DataB" width="76">260,394</td>
<td class="DataB" width="76">293,916</td>
<td class="DataB" width="76">289,545</td>
<td class="DataB" width="76">230,410</td>
<td class="DataB" width="76">NA</td>
<td class="Current2" width="76">468,347</td>
<td class="DataHist" width="76"><a class="Hist" href="./hist/n9040us2a.htm">1936-2018</a></td>
</tr>

In [18]:
name = row.find(class_='DataStub1').text
name.strip()

'U.S.'

In [20]:
#CSS for historical data
link = row.find('a')['href'][1:]
main_url_ng = 'https://www.eia.gov/dnav/ng'
link = main_url_ng + link
link

'https://www.eia.gov/dnav/ng/hist/n9040us2a.htm'

In [22]:
#Gather information from sub-page
sub_page = requests.get(link)
sub_soup = BeautifulSoup(sub_page.text, 'html.parser')
sub_soup

<!DOCTYPE HTML PUBLIC '-//W3C//DTD HTML 4.01 Transitional//EN'>
 <html> <head> <meta content="IE=9" http-equiv="X-UA-Compatible"/> <title>U.S. Natural Gas Vented and Flared  (Million Cubic Feet)</title> <link href="../Styles/ng_wrapper3.css" rel="StyleSheet" type="text/css"/> <link href="../Styles/leaf_new2.css" rel="StyleSheet" type="text/css"/> <link href="/styles/Eia_sitewideF.css" rel="StyleSheet" type="text/css"> <!-- Header Script --> <script language="JavaScript" src="/styles/eia_header.js" type="text/javascript"></script> <!--/ Header Script --> <script src="/global/includes/dnavs/leaf_handler.cfm" type="text/javascript"></script> <!-- Footer Script --> <script language="JavaScript" src="/styles/eia_footer.js" type="text/javascript"></script> <!--/ Footer Script --> </link></head> <body> <script language="JavaScript" type="text/javascript">InsertEIAHeaderCode();</script> <table border="0" cellpadding="0" cellspacing="0" width="675"> <tr> <td class="K"> </td> </tr> <tr> <td heig

In [24]:
print(sub_soup.prettify())

<!DOCTYPE HTML PUBLIC '-//W3C//DTD HTML 4.01 Transitional//EN'>
<html>
 <head>
  <meta content="IE=9" http-equiv="X-UA-Compatible"/>
  <title>
   U.S. Natural Gas Vented and Flared  (Million Cubic Feet)
  </title>
  <link href="../Styles/ng_wrapper3.css" rel="StyleSheet" type="text/css"/>
  <link href="../Styles/leaf_new2.css" rel="StyleSheet" type="text/css"/>
  <link href="/styles/Eia_sitewideF.css" rel="StyleSheet" type="text/css">
   <!-- Header Script -->
   <script language="JavaScript" src="/styles/eia_header.js" type="text/javascript">
   </script>
   <!--/ Header Script -->
   <script src="/global/includes/dnavs/leaf_handler.cfm" type="text/javascript">
   </script>
   <!-- Footer Script -->
   <script language="JavaScript" src="/styles/eia_footer.js" type="text/javascript">
   </script>
   <!--/ Footer Script -->
  </link>
 </head>
 <body>
  <script language="JavaScript" type="text/javascript">
   InsertEIAHeaderCode();
  </script>
  <table border="0" cellpadding="0" cellspac

In [27]:
# Filter for table data
data = sub_soup.find_all(class_='B3')
n = len(data)
all_data = [sub_soup.find_all(class_ = 'B3')[i].text.strip() for i in range(len(data))]
all_data

['',
 '',
 '',
 '',
 '',
 '',
 '392,528',
 '526,159',
 '649,106',
 '677,311',
 '655,967',
 '630,212',
 '626,782',
 '684,115',
 '1,010,285',
 '896,208',
 '1,102,033',
 '1,067,938',
 '810,178',
 '853,884',
 '801,044',
 '793,186',
 '848,608',
 '810,276',
 '723,567',
 '773,639',
 '864,334',
 '809,148',
 '633,412',
 '571,048',
 '562,877',
 '523,533',
 '425,629',
 '383,408',
 '341,853',
 '319,143',
 '375,695',
 '489,877',
 '516,508',
 '525,750',
 '489,460',
 '284,561',
 '248,119',
 '248,292',
 '169,381',
 '133,913',
 '131,930',
 '136,807',
 '153,350',
 '167,019',
 '125,451',
 '98,017',
 '93,365',
 '94,962',
 '107,913',
 '94,778',
 '97,633',
 '123,707',
 '142,525',
 '141,642',
 '150,415',
 '169,909',
 '167,519',
 '226,743',
 '228,336',
 '283,739',
 '272,117',
 '256,351',
 '103,019',
 '110,285',
 '91,232',
 '96,913',
 '99,178',
 '98,113',
 '96,408',
 '119,097',
 '129,469',
 '143,457',
 '166,909',
 '165,360',
 '165,928',
 '209,439',
 '212,848',
 '260,394',
 '293,916',
 '289,545',
 '230,410',
 '

In [ ]:
#Pull everything together for natural gas flare volume for each state
url = 'https://www.eia.gov/dnav/ng/ng_prod_sum_a_EPG0_VGV_mmcf_a.htm'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
#year = [y for y in range(1930, 2020)]
for row in soup.find_all('tr', class_ = 'DataRow'):
    #State
    state = row.find(class_='DataStub1').text
    #Sublink for historical data
    sub_link = row.find('a')['href'][1:]
    sub_link = url[ : 27] + "'" + sub_link
    sub_page = requests.get(link)
    sub_soup = BeautifulSoup(sub_page.text, 'html.parser')
    data = sub_soup.find_all(class_='B3')
    n = len(data)
    all_data = [sub_soup.find_all(class_ = 'B3')[i].text.strip() for i in range(len(data))]
    #out = zip(year, all_data)
    print([state, all_data])
    time.sleep(1.0)

    
    

['U.S.', ['', '', '', '', '', '', '392,528', '526,159', '649,106', '677,311', '655,967', '630,212', '626,782', '684,115', '1,010,285', '896,208', '1,102,033', '1,067,938', '810,178', '853,884', '801,044', '793,186', '848,608', '810,276', '723,567', '773,639', '864,334', '809,148', '633,412', '571,048', '562,877', '523,533', '425,629', '383,408', '341,853', '319,143', '375,695', '489,877', '516,508', '525,750', '489,460', '284,561', '248,119', '248,292', '169,381', '133,913', '131,930', '136,807', '153,350', '167,019', '125,451', '98,017', '93,365', '94,962', '107,913', '94,778', '97,633', '123,707', '142,525', '141,642', '150,415', '169,909', '167,519', '226,743', '228,336', '283,739', '272,117', '256,351', '103,019', '110,285', '91,232', '96,913', '99,178', '98,113', '96,408', '119,097', '129,469', '143,457', '166,909', '165,360', '165,928', '209,439', '212,848', '260,394', '293,916', '289,545', '230,410', 'NA', '468,347', '']]
['Alaska', ['', '', '', '', '', '', '392,528', '526,159',

In [7]:
#Put everything together for oil production, gas production, and gas flare
main_oil_url = 'https://www.eia.gov/dnav/pet/pet_crd_crpdn_adc_mbbl_m.htm'
main_ngf_url = 'https://www.eia.gov/dnav/ng/ng_prod_sum_a_EPG0_VGV_mmcf_a.htm'
main_ngp_url = 'https://www.eia.gov/dnav/ng/ng_prod_sum_a_EPG0_FGW_mmcf_a.htm'
pages = [main_oil_url, main_ngp_url, main_ngf_url]

for url in pages:
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    for row in soup.find_all('tr', class_ = 'DataRow'):
        #State
        state = row.find(class_='DataStub1').text
        #Sublink for historical data
        sub_link = row.find('a')['href'][1:]
        sub_link = url[ : 27] + "'" + sub_link
        sub_page = requests.get(sub_link)
        sub_soup = BeautifulSoup(sub_page.text, 'html.parser')
        data = sub_soup.find_all(class_='B3')
        n = len(data)
        all_data = [sub_soup.find_all(class_ = 'B3')[i].text.strip() for i in range(len(data))]
        #out = zip(year, all_data)
        print([state, all_data])
        time.sleep(1.0)
    time.sleep(20)
    

['U.S.', []]
['PADD 1', []]
['Florida', []]
['New York', []]
['Pennsylvania', []]
['Virginia', []]
['West Virginia', []]
['PADD 2', []]
['Illinois', []]
['Indiana', []]
['Kansas', []]
['Kentucky', []]
['Michigan', []]
['Missouri', []]
['Nebraska', []]


KeyboardInterrupt: 